In [40]:
#Input
word = "cover"
word1 = "cover.02"
word2 = "CAUSE-COVER"

In [41]:
import jsontrips
import xml.etree.ElementTree as ET
ont = jsontrips.ontology()
def trips_roles(onttype):
    roles = []
    for role_dic in ont[onttype]['arguments']:
        roles.append([role_dic['role'] , role_dic['optionality']])
    return roles
def parse(root , rolesetid):
    if root == None:
        return
    for predicate in root:
        for roleset in predicate:
            if roleset.attrib['id'] == rolesetid:
                roles = extract_roles(roleset)
                #vnroles = extract_vnroles(roleset)
                break
        if roleset.attrib['id'] == rolesetid:
            break
    return roles
def extract_roles(roleset):
    roles_list = []
    vnroles = []
    for things in roleset:
        if things.tag == "roles":
            for role_thing in things:
                if role_thing.tag == "role":
                    roles_list.append([role_thing.attrib['n'],role_thing.attrib['f']])
                    vnr = []
                    for vn in role_thing:
                        if vn.tag == 'vnrole':
                            vnr.append(vn.attrib['vntheta'])
                    vnroles.append(vnr)
            break
    return roles_list , vnroles

In [42]:
tree = ET.parse( word + ".xml")
root = tree.getroot()
a, b = parse(root , word1)
c = trips_roles(word2)

In [43]:
a

[['0', 'pag'], ['1', 'ppt'], ['2', 'ppt']]

In [44]:
b

[['agent'], ['destination', 'co-theme'], ['theme', 'theme']]

In [45]:
c

[['AFFECTED1', 'OPTIONAL'],
 ['AFFECTED-RESULT', 'ESSENTIAL'],
 ['RESULT', 'OPTIONAL'],
 ['AFFECTED', 'ESSENTIAL'],
 ['AGENT', 'ESSENTIAL']]

In [46]:
def role_matching(a,b,c):
    if a[0][0] == '0':
        flag1 = 0
        for tuples in c:
            if tuples[0] == 'AGENT':
                flag1 = 1
            if tuples[0] == 'EXPERIENCER':
                flag1 = -1
        if flag1 == 1:
            print("ARG0 matched to AGENT")
        elif flag1 == -1:
            print("ARG0 matched to EXPERIENCER")
        else:
            print("ARG0 is unmatched")

In [47]:
role_matching(a,b,c)

ARG0 matched to AGENT


In [55]:
def present(num , a):
    for i in range(len(a)):
        if a[i][0] == num:
            return (True , i)
    return (False , 0)

In [49]:
def remove(listi , word):
    for i in range(len(listi)):
        if listi[i] == word:
            del listi[i]
            break

In [85]:
def from_vn(num , role , vnrole):
    dic = {'agent':['AGENT','EXPERIENCER'],'causer':['AGENT'],'instrument':['AGENT'],'stimulus':['AGENT','NEUTRAL','FORMAL'],
           'theme':['AGENT','AFFECTED','NEUTRAL','FORMAL'],'patient':['AFFECTED'],'destination':['AFFECTED'],'source':['AFFECTED'],
           'experiencer':['AFFECTED','NEUTRAL','EXPERIENCER'],'product':['AFFECTED'],'result':['AFFECTED'],'pivot':['AFFECTED'],
           'location':['NEUTRAL'],'topic':['NEUTRAL'],'attribute':['FORMAL'],'predicate':['FORMAL'],'result':['FORMAL'],
          }
    if num == 1:
        if role == 'EXPERIENCER':
            l = dic[vnrole]
            remove(l,'EXPERIENCER')
            remove(l,'AGENT')
            remove(l,'AFFECTED')
            return l
        elif role == 'AGENT':
            l = dic[vnrole]
            remove(l,'EXPERIENCER')
            remove(l,'AGENT')
            return l

In [89]:
def role_matching(a,b,c):
    # Match ARG0 if present
    if present('0',a)[0]:
        if present('EXPERIENCER' , c)[0]:
            e = 'EXPERIENCER'
            print("ARG0 matched to EXPERIENCER")
        elif present('AGENT' , c)[0]:
            e = 'AGENT'
            print("ARG0 matched to AGENT")
        else:
            print("ARG0 is unmatched")
    if present('1',a)[0]:
        l = from_vn(1 , e , b[present('1',a)[1]][0])
        print("ARG1 is matched to {}".format(l))        
    #if present('2',a):
        # DO STUFF
    #if present('3',a):
        # DO STUFF

In [90]:
role_matching(a,b,c)

ARG0 matched to AGENT
ARG1 is matched to ['AFFECTED']
